In [1]:
%%capture
!pip install pygame
!pip install pygad

!git clone https://github.com/karinemiras/evoman_framework.git tmp
!cp -r /kaggle/working/tmp/* /kaggle/working/
!rm -R /kaggle/working/tmp

In [2]:
import sys, os
import numpy as np
import pandas as pd

import random

from evoman.environment import Environment
from evoman.controller import Controller
from demo_controller import player_controller
from deap import base, creator, tools, algorithms
import multiprocessing
import optuna

import seaborn as sns
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
class individual:
    
    def __init__(self, initial_range, individual_size):
        self.genes = np.random.uniform(low=-initial_range,high=initial_range,size=(individual_size))
        self.evaluated = False
        self.fitness = np.zeros((8))
        self.player_energy = np.zeros((8))
        self.enemy_energy = np.zeros((8))
        
class log_book():
    
    def __init__(self):
        self.logs = []
        
    def create_log(self,pop,gen,nevals,train_enemies,hof_gain,hof_defeat):
        log = {}
        train_gains = np.array([sum(ind.gains[train_enemies-1]) for ind in pop])
        all_gains = np.array([ind.sum_gain for ind in pop])
        all_defeated = np.array([ind.defeated for ind in pop])
        log['gen'] = gen
        log['nevals'] = nevals
        log['train_enemies_gain_mean'] = train_gains.mean()
        log['train_enemies_gain_max'] = train_gains.max()
        log['train_enemies_gain_std'] = train_gains.std()
        log['all_enemies_gain_mean'] = all_gains.mean()
        log['all_enemies_gain_max'] = all_gains.max()
        log['all_enemies_defeated_max'] = all_defeated.max()
        log['hof_gain_max'] = hof_gain['max_value']
        log['hof_defeat_max'] = hof_defeat['max_value']
        self.logs.append(log)
        self.pd = pd.DataFrame(self.logs)
        
def play(x):
    global env
    if x.evaluated:
        return x
    else:
        for enemy in range(1,9):
            env.update_parameter("enemies",[enemy])
            res = env.play(x.genes)
            x.fitness[enemy-1] = res[0]
            x.player_energy[enemy-1] = res[1]
            x.enemy_energy[enemy-1] = res[2]
        x.gains = (x.player_energy-x.enemy_energy)
        x.sum_gain = x.gains.sum()
        x.defeated = (x.enemy_energy==0).sum()
        x.evaluated = True
        return x

In [4]:
def parameter_count(hidden_neurons):
    if hidden_neurons>0:
        n_w = (20*hidden_neurons) + (hidden_neurons*5)
        n_b = hidden_neurons + 5
        return n_w + n_b
    else:
        return (20*5)+5

In [5]:
def cxWholeAritmetic(parent1,parent2,alpha):
    child1,child2 = individual(0,len(parent1.genes)), individual(0,len(parent1.genes))
    child1.genes = (alpha*parent1.genes)+((1-alpha)*parent2.genes)
    child2.genes = (alpha*parent2.genes)+((1-alpha)*parent1.genes)
    return child1,child2

def cxBlend(parent1,parent2,alpha):
    child1,child2 = individual(0,len(parent1.genes)), individual(0,len(parent1.genes))
    gamma = (1+2*alpha)*np.random.uniform(size=(len(parent1.genes,))) - alpha
    child1.genes = (gamma*parent1.genes)+((1-gamma)*parent2.genes)
    child2.genes = (gamma*parent2.genes)+((1-gamma)*parent1.genes)
    return child1, child2

In [6]:
def mutGaussian(ind,sigma,indpb):
    new_ind = individual(0,len(ind.genes))
    mutation = [np.random.normal(0,sigma) if np.random.uniform()<indpb else 0 for i in range(len(ind.genes))]
    new_ind.genes = ind.genes + np.array(mutation)
    return new_ind

def mutGaussianAdaptive(ind,sigma,tau,indpb):
    new_ind = individual(0,len(ind.genes))
    new_sigma = sigma * np.exp(np.random.normal(0,tau,size=(len(ind.genes),)))
    new_sigma[new_sigma<0.1] = 0.1
    mutation = [np.random.normal(0,s) if np.random.uniform()<indpb else 0 for s in new_sigma]
    new_ind.genes = ind.genes + np.array(mutation)
    return new_ind

def selTournament(individuals,k,tourn_size,train_enemies):
    selected = []
    for i in range(k):
        tourn_selection = np.random.choice(individuals,tourn_size)
        best = np.argmax([ind.gains[train_enemies-1].sum() for ind in tourn_selection])
        selected.append(tourn_selection[best])
    return selected

In [7]:
def genetic_algorithm(train_enemies,initial_range,pop_size,n_offspring,tournsize_parent,cxpb,cx_type,
                      alpha,mutpb,mut_type,sigma,indpb,tau,tournsize_survival,ngen,verbose):
    
    global individual_size,env

    train_enemies = np.array(train_enemies)
    pop = [individual(initial_range,individual_size) for i in range(pop_size)]
    hof_gain = {'best_solution':-np.inf,'max_value':-np.inf}
    hof_defeat = {'best_solution':-np.inf,'max_value':-np.inf}
    logbook = log_book()

    with multiprocessing.Pool() as pool:
        # evaluate initial population
        pop = list(pool.map(play,pop))

        # record best solution
        for ind in pop:
            if ind.sum_gain>hof_gain['max_value']:
                hof_gain['best_solution'] = ind
                hof_gain['max_value'] = ind.sum_gain
            if ind.defeated>hof_defeat['max_value']:
                hof_defeat['best_solution'] = ind
                hof_defeat['max_value'] = ind.defeated
        
        logbook.create_log(pop,0,len(pop),train_enemies,hof_gain,hof_defeat)
        print("gen: 0")

        for g in range(ngen):
            # select parents
            parents = selTournament(individuals=pop,k=n_offspring*pop_size,tourn_size=tournsize_parent,train_enemies=train_enemies)

            # create offspring
            offspring = []
            for child1, child2 in zip(parents[::2], parents[1::2]):
                if np.random.uniform() < cxpb:
                    if cx_type=="whole_aritmetic":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                        else:
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                    elif cx_type=="blend":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxBlend(child1, child2, alpha)
                        else:
                            child1,child2 = cxBlend(child1, child2, alpha)
                offspring.append(child1)
                offspring.append(child2)

            # apply mutation to offspring
            for i in range(len(offspring)):
                if np.random.uniform() < mutpb:
                    if mut_type=="normal":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                        else:
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                    elif mut_type=="adaptive":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)
                        else:
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)        

            # evaluate offspring
            invalid_ind = [ind for ind in offspring if not ind.evaluated]
            offspring = list(pool.map(play,offspring))

            # record best solution
            for ind in offspring:
                if ind.sum_gain>hof_gain['max_value']:
                    hof_gain['best_solution'] = ind
                    hof_gain['max_value'] = ind.sum_gain
                if ind.defeated>hof_defeat['max_value']:
                    hof_defeat['best_solution'] = ind
                    hof_defeat['max_value'] = ind.defeated

            # select the next generation of the population
            pop = selTournament(individuals=offspring,k=pop_size,tourn_size=tournsize_survival,train_enemies=train_enemies)

            logbook.create_log(pop,g+1,len(invalid_ind),train_enemies,hof_gain,hof_defeat)
            print("gen: "+str(g+1))
        pool.close()
        pool.join()

    display(logbook.pd)
    return hof_gain,hof_defeat,logbook
    

In [8]:
hidden_neurons = 10
individual_size = parameter_count(hidden_neurons)

headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

env = Environment(playermode="ai",
                  player_controller=player_controller(hidden_neurons),
                  speed="fastest",
                  enemymode="static",
                  level=2,
                  visuals=False,
                  logs="off")

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5704:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default


In [15]:
def objective(trial):
    
    params = {'train_enemies':trial.suggest_categorical("train_enemies",[[7,8],[2,5,6],[1,2,6,7]]),
              'initial_range':trial.suggest_int("initial_range", 1,3,step=1),
              'pop_size':trial.suggest_int("pop_size", 50,50,step=50),
              'n_offspring':trial.suggest_int("n_offspring", 1,5,step=1),
              'tournsize_parent':trial.suggest_int("tournsize_parent", 1,10,step=1),
              'cxpb':trial.suggest_float("cxpb", 0.1, 1., step=0.1),
              'cx_type':trial.suggest_categorical("cx_type",["whole_aritmetic","blend"]),
              'alpha':trial.suggest_categorical("alpha",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'mutpb':trial.suggest_float("mutpb", 0.1, 1., step=0.1),
              'mut_type':trial.suggest_categorical("mut_type",["normal","adaptive"]),
              'sigma':trial.suggest_float("sigma", 0.5, 3, step=0.5),
              'indpb':trial.suggest_categorical("indpb",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'tau':trial.suggest_float("tau", 0.1, 1., step=0.1),
              'tournsize_survival':trial.suggest_int("tournsize_survival", 1,10,step=1),
              'ngen':trial.suggest_int("ngen", 30,30,step=30),
              'verbose':trial.suggest_int("verbose", 1,1,step=1)}
    
    global hof_tuning_gain,hof_tuning_defeat
    iteration_logs = []
    for i in range(3):
        hof_gain,hof_defeat,logs = genetic_algorithm(**params)
        logs.pd['iteration'] = i+1
        iteration_logs.append(logs.pd)
        hof_tuning_gain.append(hof_gain)
        hof_tuning_defeat.append(hof_defeat)
    
    return pd.concat(iteration_logs).groupby('iteration')['all_enemies_gain_max'].agg(lambda x: np.mean(np.unique(x)[-5:])).mean()


In [16]:
hof_tuning_gain,hof_tuning_defeat = [], []

study = optuna.create_study(study_name="GA", direction="maximize")

study.optimize(objective, n_trials=500)

[I 2023-10-14 23:58:56,622] A new study created in memory with name: GA
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [7, 8] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [2, 5, 6] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 6, 7] which is of type list.
  warnings.warn(message)


gen: 0
gen: 1
gen: 2
gen: 3
gen: 4
gen: 5
gen: 6
gen: 7
gen: 8
gen: 9
gen: 10
gen: 11
gen: 12
gen: 13
gen: 14
gen: 15
gen: 16
gen: 17
gen: 18
gen: 19
gen: 20
gen: 21
gen: 22
gen: 23
gen: 24
gen: 25
gen: 26
gen: 27
gen: 28
gen: 29
gen: 30


,gen,nevals,train_enemies_gain_mean,train_enemies_gain_max,train_enemies_gain_std,all_enemies_gain_mean,all_enemies_gain_max,all_enemies_defeated_max,hof_gain_max,hof_defeat_max
0,0,50,-182.236,66.2,50.123985,-701.000,-175.4,4,-175.4,4
1,1,44,-8.232,66.2,78.557305,-341.056,-175.4,4,-175.4,4
2,2,42,55.800,66.2,24.995616,-208.964,-175.4,4,-175.4,4
3,3,41,66.008,81.2,11.429923,-197.960,-175.4,4,-155.4,4
4,4,42,68.592,88.4,34.054943,-252.488,-166.0,4,-155.4,4
5,5,47,75.416,88.4,34.917018,-204.036,-172.4,4,-155.4,4
6,6,41,77.944,88.4,31.984422,-188.836,-79.8,4,-79.8,4
7,7,47,53.648,88.4,43.722188,-246.192,-172.4,4,-79.8,4
8,8,41,85.096,88.4,16.205986,-179.480,-172.4,4,-79.8,4
9,9,41,79.884,88.4,31.442852,-193.872,-172.4,4,-79.8,4


gen: 0
gen: 1
gen: 2
gen: 3
gen: 4
gen: 5
gen: 6
gen: 7
gen: 8
gen: 9
gen: 10
gen: 11
gen: 12
gen: 13
gen: 14
gen: 15
gen: 16
gen: 17
gen: 18
gen: 19
gen: 20
gen: 21
gen: 22
gen: 23
gen: 24
gen: 25
gen: 26
gen: 27
gen: 28
gen: 29
gen: 30


,gen,nevals,train_enemies_gain_mean,train_enemies_gain_max,train_enemies_gain_std,all_enemies_gain_mean,all_enemies_gain_max,all_enemies_defeated_max,hof_gain_max,hof_defeat_max
0,0,50,-192.400,-120.0,19.241622,-716.488,-311.6,1,-311.6,1
1,1,41,-71.176,29.0,68.131856,-411.644,-219.4,2,-219.4,2
2,2,45,25.576,29.0,10.475620,-411.364,-332.4,3,-219.4,3
3,3,44,17.456,31.4,31.975692,-397.136,-200.2,3,-165.8,3
4,4,42,24.388,31.4,21.930998,-358.300,-283.6,2,-149.6,3
5,5,42,34.132,93.8,35.492030,-329.972,-139.0,4,-139.0,4
6,6,42,68.484,94.4,44.303229,-306.720,-170.6,4,-139.0,4
7,7,43,78.208,94.4,35.804915,-286.556,-256.0,3,-139.0,4
8,8,45,78.104,94.4,46.180069,-286.612,-235.6,2,-139.0,4
9,9,43,77.432,94.4,37.261441,-271.760,-115.8,4,-115.8,4


...

,gen,nevals,train_enemies_gain_mean,train_enemies_gain_max,train_enemies_gain_std,all_enemies_gain_mean,all_enemies_gain_max,all_enemies_defeated_max,hof_gain_max,hof_defeat_max
0,0,50,-172.844,78.2,60.247294,-688.816,-343.8,3,-343.8,3
1,1,47,-9.080,67.4,60.001587,-371.916,-184.0,3,-184.0,3
2,2,45,27.228,67.4,40.058977,-323.444,-184.0,3,-184.0,3
3,3,38,57.736,75.8,25.119250,-214.780,-184.0,3,-184.0,3
4,4,45,59.756,90.2,35.000807,-250.788,-141.8,4,-141.8,4
5,5,39,81.420,90.2,22.880463,-202.256,-141.8,4,-141.8,4
6,6,44,64.856,90.2,64.933604,-225.228,-141.8,4,-141.8,4
7,7,40,86.876,90.8,14.842797,-152.528,-141.8,4,-141.8,4
8,8,40,70.456,90.8,42.231041,-189.864,-141.8,4,-141.8,4
9,9,46,76.580,90.8,27.647937,-203.804,-166.8,4,-141.8,4


[I 2023-10-15 00:05:08,287] Trial 0 finished with value: -95.49333333333254 and parameters: {'train_enemies': [7, 8], 'initial_range': 1, 'pop_size': 50, 'n_offspring': 1, 'tournsize_parent': 8, 'cxpb': 0.30000000000000004, 'cx_type': 'blend', 'alpha': 0.30000000000000004, 'mutpb': 0.8, 'mut_type': 'adaptive', 'sigma': 3.0, 'indpb': 0.9, 'tau': 0.1, 'tournsize_survival': 7, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: -95.49333333333254.
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [7, 8] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [2, 5, 6] which is of type list.
  warnings.warn(message)
/opt/conda/lib/py

gen: 0
gen: 1
gen: 2
gen: 3
gen: 4
gen: 5
gen: 6
gen: 7
gen: 8
gen: 9
gen: 10
gen: 11
gen: 12
gen: 13
gen: 14


Process ForkPoolWorker-41:
Process ForkPoolWorker-44:
Process ForkPoolWorker-42:
Process ForkPoolWorker-43:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result =

KeyboardInterrupt: 

In [ ]:
"""
params = {'train_enemies':[7,8],
          'initial_range':1,
          'pop_size':50,
          'n_offspring':2,
          'tournsize_parent':5,
          'cxpb':0.7,
          'cx_type':'blend',
          'alpha':'random',
          'mutpb':0.3,
          'mut_type':'adaptive',
          'sigma':2,
          'indpb':0.8,
          'tau':0.3,
          'tournsize_survival':10,
          'ngen':10,
          'verbose':1}
          
""""